In [5]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import matplotlib.pyplot as plt
from cassandra.cluster import Cluster
import requests


In [6]:
!pip install cassandra-driver




In [24]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [8]:
keyspace_name = "data_quran"  


session.execute("CREATE KEYSPACE IF NOT EXISTS data_quran WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}")

session.set_keyspace('data_quran')


In [9]:

session.set_keyspace('data_quran')

In [10]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS arabic (
        verse_number int PRIMARY KEY,
        surah int,
        ayah_id int,
        text varchar
    );
"""

try:
    session.execute(create_table_query)
    print("Table 'arabic' created successfully!")
except Exception as e:
    print("Error:", e)

Table 'arabic' created successfully!


In [11]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS Tafseer (
        Name varchar,
        Surah int,
        Ayat int,
        Verse text,
        Tafseer text,
        PRIMARY KEY (Name, Surah, Ayat)
    );
"""

try:
    session.execute(create_table_query)
    print("Table 'Tafseer' created successfully!")
except Exception as e:
    print("Error: Issue creating table")
    print(e)

Table 'Tafseer' created successfully!


In [12]:
csv_file = r"C:\Users\ASUS\Documents\GitHub\mogodb\big-data-engineering-with-python-2023\popopo\arabic_ayah.csv"  # Replace with your CSV file path

try:
    df = pd.read_csv(csv_file)
    
    for row in df.itertuples(index=False):
        insert_query = session.prepare("INSERT INTO arabic (verse_number, surah, ayah_id, text) VALUES (?, ?, ?, ?)")
        session.execute(insert_query, (row.verse_number, row.surah, row.ayah_id, row.text))
        
    print("Data inserted successfully!")
except Exception as e:
    print("Error: Inserting Rows")
    print(e)

Data inserted successfully!


In [13]:
file_path = r"C:\Users\ASUS\Documents\GitHub\mogodb\big-data-engineering-with-python-2023\popopo\Quran_English_with_Tafseer.csv"

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        next(file) 
        for line in file:
            parts = line.strip().split(',')  
            name = parts[0]
            surah = int(parts[1])
            ayat = int(parts[2])
            verse = parts[3]
            tafseer = parts[4]
            insert_query = session.prepare("INSERT INTO Tafseer (Name, Surah, Ayat, Verse, Tafseer) VALUES (?, ?, ?, ?, ?)")
            session.execute(insert_query, (name, surah, ayat, verse, tafseer))
    print("Data inserted successfully!")
except Exception as e:
    print("Error: Inserting Data")
    print(e)

Data inserted successfully!


In [46]:
keyspace_name = "data_quran"
session.set_keyspace(keyspace_name)


table_name = "arabic"


rows = session.execute(f"SELECT * FROM {table_name} LIMIT 300")


for row in rows:
    print(row)  

Row(verse_number=4317, ayah_id=45, surah=42, text='وَتَرَىٰهُمۡ يُعۡرَضُونَ عَلَيۡهَا خَٰشِعِينَ مِنَ ٱلذُّلِّ يَنظُرُونَ مِن طَرۡفٍ خَفِيّٖۗ وَقَالَ ٱلَّذِينَ ءَامَنُوٓاْ إِنَّ ٱلۡخَٰسِرِينَ ٱلَّذِينَ خَسِرُوٓاْ أَنفُسَهُمۡ وَأَهۡلِيهِمۡ يَوۡمَ ٱلۡقِيَٰمَةِۗ أَلَآ إِنَّ ٱلظَّـٰلِمِينَ فِي عَذَابٖ مُّقِيمٖ')
Row(verse_number=3372, ayah_id=32, surah=29, text='قَالَ إِنَّ فِيهَا لُوطٗاۚ قَالُواْ نَحۡنُ أَعۡلَمُ بِمَن فِيهَاۖ لَنُنَجِّيَنَّهُۥ وَأَهۡلَهُۥٓ إِلَّا ٱمۡرَأَتَهُۥ كَانَتۡ مِنَ ٱلۡغَٰبِرِينَ')
Row(verse_number=1584, ayah_id=111, surah=11, text='وَإِنَّ كُلّٗا لَّمَّا لَيُوَفِّيَنَّهُمۡ رَبُّكَ أَعۡمَٰلَهُمۡۚ إِنَّهُۥ بِمَا يَعۡمَلُونَ خَبِيرٞ')
Row(verse_number=4830, ayah_id=46, surah=53, text='مِن نُّطۡفَةٍ إِذَا تُمۡنَىٰ')
Row(verse_number=2731, ayah_id=58, surah=23, text='وَٱلَّذِينَ هُم بِـَٔايَٰتِ رَبِّهِمۡ يُؤۡمِنُونَ')
Row(verse_number=5056, ayah_id=77, surah=56, text='إِنَّهُۥ لَقُرۡءَانٞ كَرِيمٞ')
Row(verse_number=2713, ayah_id=40, surah=23, text='قَالَ عَمَّا قَلِيلٖ

In [52]:
keyspace_name = "data_quran"
session.set_keyspace(keyspace_name)


table_name = "Tafseer"


rows = session.execute(f"SELECT * FROM {table_name} LIMIT 400")


for row in rows:
    print(row) 
    

Row(name='The Reality', surah=69, ayat=1, tafseer='The Reality the Resurrection in which is realised the truth of all that was rejected in the way of the raising from the graves the reckoning and the requital or it means the Resurrection which will manifest all of that.', verse='The Reality!')
Row(name='The Reality', surah=69, ayat=2, tafseer='What is the Reality? ma’l-hāqqa an interrogative to emphasise its enormity; this is the subject as well as the predicate of the previous al-hāqqa ‘the Reality’.', verse='What is the Reality ?')
Row(name='The Reality', surah=69, ayat=3, tafseer=' what will convey unto thee what the reality is!"', verse='"Ah')
Row(name='The Reality', surah=69, ayat=4, tafseer='Thamūd and ‘Ād denied the Clatterer the Resurrection because its terrors cause the hearts to clatter.', verse='(The tribes of) Thamud and A’ad disbelieved in the judgment to come.')
Row(name='The Reality', surah=69, ayat=5, tafseer=' they were destroyed by the lightning."', verse='"As for Tha

Question

1 How many the phrase إِنَّ in quran?

In [41]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="إِنَّ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in al-quran.")

The phrase 'إِنَّ' occurs 1362 times in al-quran.


2 How many the phrase هُم in quran?

In [29]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="هُم"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in al-quran.")

The phrase 'هُم' occurs 1899 times in al-quran.


3 How many word 'say' appears in al-quran?

In [34]:
query = "SELECT surah,ayat, tafseer From Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []
    
search_word = "say"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times.")

The word 'say' appears 461 times.


4 How many the phrase لَا in quran?

In [45]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="لَا"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in al-quran.")

The phrase 'لَا' occurs 1364 times in al-quran.


5 How many the phrase كُلِّ in quran?

In [39]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="كُلِّ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in al-quran.")

The phrase 'كُلِّ' occurs 185 times in al-quran.


6  How many the phrase هَلۡ in quran?

In [44]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="هَلۡ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in al-quran.")

The phrase 'هَلۡ' occurs 86 times in al-quran.


7  How many the phrase إِذَا in quran?

In [47]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="إِذَا"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in al-quran.")

The phrase 'إِذَا' occurs 371 times in al-quran.


8  How many the phrase مُنزِلِينَ in quran?

In [48]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="مُنزِلِينَ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in al-quran.")

The phrase 'مُنزِلِينَ' occurs 3 times in al-quran.


9  How many the phrase أُوْلَـٰٓئِكَ in quran?

In [49]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="أُوْلَـٰٓئِكَ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in al-quran.")

The phrase 'أُوْلَـٰٓئِكَ' occurs 187 times in al-quran.


10  How many the phrase عَظِيمٗا in quran?

In [50]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="عَظِيمٗا"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in al-quran.")

The phrase 'عَظِيمٗا' occurs 17 times in al-quran.


11  How many the phrase جَمۡعًا in quran?

In [51]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="جَمۡعًا"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in al-quran.")

The phrase 'جَمۡعًا' occurs 1 times in al-quran.


12 How many word 'Moses' appears in al-quran?

In [53]:
query = "SELECT surah,ayat, tafseer From Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []
    
search_word = "Moses"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times.")

The word 'Moses' appears 72 times.


13 How many word 'me' appears in al-quran?

In [54]:
query = "SELECT surah,ayat, tafseer From Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []
    
search_word = "me"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times.")

The word 'me' appears 1739 times.


14 How many word 'Muhammad' appears in al-quran?

In [55]:
query = "SELECT surah,ayat, tafseer From Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []
    
search_word = "Muhammad"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times.")

The word 'Muhammad' appears 121 times.


15 How many word 'enjoin' appears in al-quran?

In [60]:
query = "SELECT surah,ayat, tafseer From Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []
    
search_word = "enjoin"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times.")

The word 'enjoin' appears 7 times.


16 How many word 'Adam' appears in al-quran?

In [61]:
query = "SELECT surah,ayat, tafseer From Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []
    
search_word = "Adam"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times.")

The word 'Adam' appears 13 times.


17 How many word 'angels' appears in al-quran?

In [62]:
query = "SELECT surah,ayat, tafseer From Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []
    
search_word = "angels"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times.")

The word 'angels' appears 65 times.


18 How many word 'pens' appears in al-quran?

In [63]:
query = "SELECT surah,ayat, tafseer From Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []
    
search_word = "pens"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times.")

The word 'pens' appears 13 times.


19 How many word 'hidden' appears in al-quran?

In [64]:
query = "SELECT surah,ayat, tafseer From Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []
    
search_word = "hidden"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times.")

The word 'hidden' appears 23 times.


20 How many word 'messenger' appears in al-quran?

In [65]:
query = "SELECT surah,ayat, tafseer From Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []
    
search_word = "messenger"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1


print(f"The word '{search_word}' appears {occurrence_count} times.")

The word 'messenger' appears 135 times.
